# Likelihood Optimization of gas Kinematics in IFUs (LOKI)
## Fitting example: NIRSpec + aperture model

Michael Reefe

This example notebook provides a tutorial on how to run LOKI on a single-channel NIRSpec IFU cube (F170LP/G235H), fitting an integrated spectrum within an aperture.

In [1]:
using Pkg
Pkg.activate(dirname(@__DIR__))
Pkg.instantiate()
Pkg.precompile()
using Loki
using Unitful 

  Activating project at `~/Dropbox/Astrophysics/Phoenix_Cluster/Loki`
Precompiling packages...
  11237.6 ms  ✓ Loki
  1 dependency successfully precompiled in 13 seconds. 308 already precompiled.


Now we want to load in our data. For this example, we'll be using the data for NGC 7469 over the grating/filter combos G140H/F100LP, G235H/F170LP, and G395H/F290LP, which are located in the same folder as this notebook. The JWST reduced data does not include a redshift, so we must provide the redshift ourselves.  We can use the `from_fits` function to load in the JWST-formatted FITS files, along with the redshift.

In [10]:
# The redshift of the target object: NGC 7469
z = 0.016317
# The semicolon at the end suppresses printing the output Observation object, which is long and not very enlightening
obs = from_fits(["f170lp_g235h-f170lp_s3d.fits"], z);

[ Info: Initializing DataCube struct from f170lp_g235h-f170lp_s3d.fits


Next, we create some variables that we will use later. We will be fitting data from multiple gratings/filters, and we can take the `name` property from the Observation object we just loaded in to get the name of the target. Here, `run_name` is just a unique identifier that we will use for this run.

In [15]:
channel = :G235H_F170LP
nm = replace(obs.name, " " => "_") 
run_name = "$(nm)_$(channel)_nuc_aperture"

"NGC_7469_G235H_F170LP_nuc_aperture"

Before fitting, we want to do some pre-processing on the data. We want to convert the data to the rest-frame, mast out / interpolate any bad pixels, and replace the JWST pipeline-generated errors with some more realistic ones.  All of this is achieved in the next block of code. This is also where we will combine data from multiple channels into a single cube using the `combine_channels!` function.

In [16]:
if isfile("$nm.channel$channel.rest_frame.fits")
    # If we've already performed this step in a previous run, just load in the pre-processed data
    obs = from_fits(["$nm.channel$channel.rest_frame.fits"], obs.z);
    
else
    # Convert to rest-frame wavelength vector, and mask out bad spaxels
    correct!(obs)

    # Reproject the sub-channels onto the same WCS grid and combine them into one full channel
    # - The [:G140H_F100LP, :G235H_F170LP, :G395H_F290LP] vector gives the names of each channel to concatenate. By default, JWST subchannels are
    #   given labels of "A" for short, "B" for medium, and "C" for long, followed by the channel number.  
    # - The "out_id" argument will determine the label given to the combined channel data. 
    # combine_channels!(obs, [:G140H_F100LP, :G235H_F170LP, :G395H_F290LP], out_id=channel, order=1, 
    #     adjust_wcs_headerinfo=true, extract_from_ap=0.)

    # rotate to the RA/Dec axes on the sky
    rotate_to_sky_axes!(obs.channels[channel])

    # We interpolate any rogue NaNs using a linear interpolation, since the MPFIT minimizer does not handle NaNs well.
    interpolate_nans!(obs.channels[channel])

    # Finally, we calculate the statistical errors (i.e. the standard deviation of the residuals with a cubic spline fit)
    # and replace the errors in the cube with these, since the provided errors are typically underestimated.
    # You can skip this step if you wish to use the default errors.
    calculate_statistical_errors!(obs.channels[channel])
    
    # Save the pre-processed data as a FITS file so it can be quickly reloaded later
    save_fits(".", obs, [channel]);
end

[ Info: Interpolating NaNs in cube with channel F170LP, band G235H:
[ Info: Calculating statistical errors for each spaxel...


Progress: 100%|█████████████████████████████████████████| Time: 0:01:21


[ Info: Writing FITS file from Observation object


We next create an aperture to define the region of interest that we would like to fit. We can do this with the `make_aperture` function. We can customize the aperture's shape, centroid, radius, etc.

In [17]:
# - The first argument is the data cube
# - The second argument is the aperture shape, which may be one of: (Circular, Rectangular, Elliptical)
# - Next are the right ascension in sexagesimal hours and the declination in sexagesimal degrees
# - The next arguments depend on the aperture shape:
#    - For circles, it is the radius in arcseconds
#    - For rectangles, it is the width in arcseconds, height in arcseconds, and rotation angle in degrees
#    - For ellipses, it is the semimajor axis in arcseconds, semiminor axis in arcseconds, and rotation angle in degrees
# - The auto_centroid argument, if true, will adjust the aperture centroid to the closest peak in brightness
# - The scale_psf argument, if true, will create a series of apertures with increasing radii that scale at the same rate as the PSF
ap = make_aperture(obs.channels[channel], :Circular, "23:03:15.610", "+8:52:26.10", 0.5, auto_centroid=true)

[ Info: Creating a circular aperture at 23:03:15.610, +8:52:26.10
[ Info: Aperture centroid adjusted to 0h56m44.36803873772078s, 8d52m26.123455876816877s


11×11 Photometry.Aperture.CircularAperture{Float64} with indices 21:31×27:37:
 0.0        0.0        0.0       …  7.09557e-5  0.0        0.0
 0.0        0.0164466  0.606929     0.669807    0.0351559  0.0
 0.0        0.555268   1.0          1.0         0.638151   0.0
 0.110852   0.984511   1.0          1.0         0.998276   0.179971
 0.37795    1.0        1.0          1.0         1.0        0.460833
 0.447792   1.0        1.0       …  1.0         1.0        0.530675
 0.314219   1.0        1.0          1.0         1.0        0.397102
 0.0403004  0.918547   1.0          1.0         0.958033   0.0836965
 0.0        0.32733    0.991953     0.998966    0.4032     0.0
 0.0        0.0        0.304628     0.360562    0.0        0.0
 0.0        0.0        0.0       …  0.0         0.0        0.0

Finally, we create the `CubeFitter` object and call the `fit_cube!` function to fit the data. Note specifically for NIRSpec data, we want to enable the option "nirspec_mask_chip_gaps".  This will make sure that when fitting any integrated regions larger than a single spaxel (i.e. apertures, voronoi bins, or the initial integrated fit over the whole FOV) the chip gap regions will be fully masked out.  This is necessary because the wavelengths of the chip gaps vary based on the position in the IFU, so when making integrated spectra, in regions where some spaxels are masked and others aren't, it can create artifical dips in the continuum.  The "nirspec_mask_chip_gaps" options masks out the maximum extent of the chip gaps over all of the IFU positions (but again, only when fitting regions larger than a single spaxel).

Also, note that since we're in the near-infrared regime, including stellar populations here is *critical*, so make sure the "fit_stellar_continuum" option is enabled!

In [18]:
# To see a full list of keyword arguments, please refer to the docstring, which can be accessed by typing `?CubeFitter` in the command
# line after importing Loki.
cube_fitter = CubeFitter(
    obs.channels[channel], 
    obs.z, 
    run_name; 
    nirspec_mask_chip_gaps=true,
    parallel=false, 
    plot_spaxels=:pyplot, 
    plot_maps=true, 
    save_fits=true,
    silicate_absorption="d+",
    extinction_screen=true, 
    use_pah_templates=false,   # <- this can pretty safely be turned off in the NIR regime since there is only really the 3.3um PAH 
    fit_sil_emission=true, 
    fit_stellar_continuum=true, 
    save_full_model=true, 
    map_snr_thresh=3., 
    subtract_cubic_spline=true,
    linemask_width=3000.0u"km/s"  # <- expand the linemask width to make sure the broad Paschen lines are covered
)

# Call the fit_cube! function on the cube_fitter object, using the aperture we defined.
fit_cube!(cube_fitter, ap)

[ Info: Preparing output directories
[ Info: Generating 400 simple stellar population templates with FSPS with ages ∈ (0.0010000000000000002 Gyr, 13.7 Gyr), log(Z/Zsun) ∈ (-2.3, 0.4)


Progress: 100%|███████████████████████████| Time: 0:15:10 ( 2.28  s/it)


┌ Info: 
│ 
│ #############################################################################
│ ######## BEGINNING FULL CUBE FITTING ROUTINE FOR NGC_7469_G235H_F170LP_nuc_aperture ########
│ #############################################################################
│ 
│ ------------------------
│ Worker Processes:     1
│ Threads per process:  1
└ ------------------------
[ Info: ===> Preparing output data structures... <===
[ Info: Performing aperture photometry to get an integrated spectrum...
[ Info: ===> Beginning integrated spectrum fitting... <===


julia(92740) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
julia(92743) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
julia(92746) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
julia(92750) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
julia(92756) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
julia(92757) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
julia(92797) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
julia(92798) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


[ Info: ===> Generating parameter maps and model cubes... <===
[ Info: ===> Writing FITS outputs... <===
┌ Info: 
│ 
│ #############################################################################
│ ################################### Done!! ##################################
└ #############################################################################


(CubeFitter{Float64, Int64, Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}, Quantity{Float64, 𝐋 𝐓⁻¹, Unitful.FreeUnits{(km, s⁻¹), 𝐋 𝐓⁻¹, nothing}}, Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}}(DataCube{Vector{Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}}, Array{Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻², s⁻¹, sr⁻¹), 𝐌 𝐓⁻², nothing}}, 3}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(μm,), 𝐋, nothing}}[1.6335434383324 μm, 1.6339330805292154 μm, 1.6343228156657337 μm, 1.634712643764123 μm, 1.6351025648465576 μm, 1.6354925789352162 μm, 1.6358826860522833 μm, 1.6362728862199485 μm, 1.636663179460407 μm, 1.637053565795859 μm  …  3.112448773992363 μm, 3.1131911732143958 μm, 3.11393374951777 μm, 3.114676502944724 μm, 3.115419433537507 μm, 3.1161625413383773 μm, 3.1169058263896035 μm, 3.117649288733465 μm, 3.1183929284122494 μm, 3.1191367454682566 μm], Quantity{Float64, 𝐌 𝐓⁻², Unitful.FreeUnits{(erg, Hz⁻¹, cm⁻²,

And the results can be found in the "output_[run_name]" directory, just like the other examples!  Here is what it looks like: 

![](./NGC7469.nirspec.nuc.aperture.png)

The orange line shows the final model.  The decomposed components of the model consist of:
- Thermal dust continuum, in gray
- Hot silicate dust emission, in light green
- Stellar continuum, in pink
- Emission lines, in purple
- Extinction, in dotted gray (read from the right axis)

The gray band shows a masked out part of the spectrum -- this is due to NIRSpec's chip gap.

Notice that some of the emission lines don't have particularly good fits.  This is because we've limited ourselves to fitting them with a single Gaussian component.  I'd recommend for fits like these setting up some additional kinematic components for the lines that need it.  This requires editing the "lines.toml" config file (check out the README file for detailed instructions on how to do this).